# llcuda v2.1.0 - Complete CUDA 12 Build for Tesla T4

**Purpose**: Build unified CUDA 12 binaries package for llcuda as Unsloth inference backend

**What This Notebook Does**:
1. Clone llama.cpp, llcuda repositories
2. Build llama.cpp with CUDA 12 + FlashAttention for Tesla T4
3. Build llcuda Python package
4. Create **ONE unified tar file** with everything
5. Download the tar file

**Output**: `llcuda-complete-cuda12-t4.tar.gz` (~350-400 MB)

**Target**:
- GPU: Tesla T4 (SM 7.5)
- CUDA: 12.x
- Platform: Google Colab
- Integration: Unsloth + llcuda

---

## Step 1: Verify Environment (Tesla T4 + CUDA 12)

In [1]:
# Check GPU - Must be Tesla T4
!nvidia-smi --query-gpu=name,compute_cap,driver_version,memory.total --format=csv

import subprocess
result = subprocess.run(
    ['nvidia-smi', '--query-gpu=name,compute_cap', '--format=csv,noheader'],
    capture_output=True, text=True
)

gpu_info = result.stdout.strip().split(',')
gpu_name = gpu_info[0].strip()
compute_cap = gpu_info[1].strip()

print(f"\n{'='*60}")
print(f"GPU: {gpu_name}")
print(f"Compute Capability: SM {compute_cap}")
print(f"{'='*60}")

if 'T4' in gpu_name and compute_cap == '7.5':
    print("\n✅ Perfect! Tesla T4 (SM 7.5) detected")
    print("   This build will be optimized for your GPU")
elif compute_cap == '7.5':
    print(f"\n⚠️  {gpu_name} (SM 7.5) - Should work but not T4")
else:
    print(f"\n❌ WARNING: SM {compute_cap} detected")
    print("   This notebook is optimized for SM 7.5 (Tesla T4)")
    print("   Build may not work optimally on your GPU")

name, compute_cap, driver_version, memory.total [MiB]
Tesla T4, 7.5, 550.54.15, 15360 MiB

GPU: Tesla T4
Compute Capability: SM 7.5

✅ Perfect! Tesla T4 (SM 7.5) detected
   This build will be optimized for your GPU


In [2]:
# Check CUDA version
!nvcc --version

import sys
print(f"\nPython: {sys.version}")
print(f"Expected: 3.10+ (Colab default)")

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Expected: 3.10+ (Colab default)


## Step 2: Install Build Dependencies

In [24]:
# Install build tools
!apt-get update -qq
!apt-get install -y -qq build-essential cmake ninja-build git curl

# Install Python build dependencies
!pip install -q pybind11 setuptools>=77.0.0 wheel build

print("\n✅ Build dependencies installed")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)

✅ Build dependencies installed


## Step 3: Clone Repositories

In [4]:
import os
os.chdir('/content')

# Clean up previous builds
!rm -rf llama.cpp llcuda

# Clone llama.cpp (for inference server)
print("\n📥 Cloning llama.cpp...")
!git clone -q https://github.com/ggerganov/llama.cpp.git
print("✅ llama.cpp cloned")






📥 Cloning llama.cpp...
✅ llama.cpp cloned

📥 Cloning llcuda v2.0.1...
✅ llcuda cloned
total 24
drwxr-xr-x  1 root root 4096 Jan 15 05:06 .
drwxr-xr-x  1 root root 4096 Jan 15 04:57 ..
drwxr-xr-x  4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 26 root root 4096 Jan 15 05:06 llama.cpp
drwxr-xr-x 11 root root 4096 Jan 15 05:06 llcuda
drwxr-xr-x  1 root root 4096 Dec  9 14:42 sample_data


In [15]:
cd ..

/content


In [18]:
ls

llama.cpp/  sample_data/


In [17]:
!rm -rf llcuda

In [19]:
# Clone llcuda (CUDA inference backend for Unsloth)
print("\n📥 Cloning llcuda v2.0.1...")
!git clone -q https://github.com/waqasm86/llcuda.git
print("✅ llcuda cloned")

!ls -la


📥 Cloning llcuda v2.0.1...
✅ llcuda cloned
total 24
drwxr-xr-x  1 root root 4096 Jan 15 06:29 .
drwxr-xr-x  1 root root 4096 Jan 15 04:57 ..
drwxr-xr-x  4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 27 root root 4096 Jan 15 06:01 llama.cpp
drwxr-xr-x 11 root root 4096 Jan 15 06:29 llcuda
drwxr-xr-x  1 root root 4096 Dec  9 14:42 sample_data


## Step 4: Build llama.cpp with CUDA 12 + FlashAttention

This builds the HTTP inference server with:
- CUDA 12 support
- FlashAttention enabled
- Tensor Core optimization for Tesla T4
- All quantization formats (Q4_K_M, Q8_0, NF4, etc.)

In [5]:
os.chdir('/content/llama.cpp')

print("\n🔨 Configuring llama.cpp for Tesla T4...")
print("   - CUDA 12.x")
print("   - SM 7.5 (Tesla T4)")
print("   - FlashAttention: ON")
print("   - CUDA Graphs: ON")
print("   - Tensor Cores: Optimized")
print("\nThis may take 1-2 minutes...\n")

!cmake -B build_t4 \
    -DCMAKE_BUILD_TYPE=Release \
    -DGGML_CUDA=ON \
    -DCMAKE_CUDA_ARCHITECTURES="75" \
    -DCMAKE_CUDA_COMPILER=/usr/local/cuda/bin/nvcc \
    -DGGML_NATIVE=OFF \
    -DGGML_CUDA_FORCE_MMQ=OFF \
    -DGGML_CUDA_FORCE_CUBLAS=OFF \
    -DGGML_CUDA_FA=ON \
    -DGGML_CUDA_FA_ALL_QUANTS=ON \
    -DGGML_CUDA_GRAPHS=ON \
    -DGGML_CUDA_PEER_MAX_BATCH_SIZE=128 \
    -DLLAMA_BUILD_SERVER=ON \
    -DLLAMA_BUILD_TOOLS=ON \
    -DLLAMA_CURL=ON \
    -DBUILD_SHARED_LIBS=ON \
    -DCMAKE_INSTALL_RPATH='$ORIGIN/../lib' \
    -DCMAKE_BUILD_WITH_INSTALL_RPATH=ON

print("\n✅ CMake configuration complete")


🔨 Configuring llama.cpp for Tesla T4...
   - CUDA 12.x
   - SM 7.5 (Tesla T4)
   - FlashAttention: ON
   - CUDA Graphs: ON
   - Tensor Cores: Optimized

This may take 1-2 minutes...

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
CMake Warning at CMakeLists.txt:121 (message):
  LLAMA_CURL option is deprecated and will be ignored


-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing 

In [6]:
import time

print("\n🔨 Building llama.cpp...")
print("   ☕ Good time for a coffee break!\n")

start = time.time()

!cmake --build build_t4 --config Release -j$(nproc)

elapsed = time.time() - start
print(f"\n✅ llama.cpp build complete in {elapsed/60:.1f} minutes")


🔨 Building llama.cpp...
   ☕ Good time for a coffee break!

[  0%] Building CXX object common/CMakeFiles/build_info.dir/build-info.cpp.o
[  0%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml.c.o
[  0%] Built target build_info
[  0%] Building CXX object vendor/cpp-httplib/CMakeFiles/cpp-httplib.dir/httplib.cpp.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml.cpp.o
[  1%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-alloc.c.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-backend.cpp.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-opt.cpp.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-threading.cpp.o
[  1%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-quants.c.o
[  2%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/gguf.cpp.o
[  2%] Linking CXX shared library ../../bin/libggml-base.so
[  2%] Built target ggml-base
[  2%] Building C object examples/gguf-hash/CMakeFiles/

In [7]:
pwd

'/content/llama.cpp'

In [8]:
ls

AGENTS.md                       gguf-py/
AUTHORS                         grammars/
benches/                        include/
build_t4/                       LICENSE
build-xcframework.sh*           licenses/
ci/                             Makefile
CLAUDE.md                       media/
cmake/                          models/
CMakeLists.txt                  mypy.ini
CMakePresets.json               pocs/
CODEOWNERS                      poetry.lock
common/                         pyproject.toml
CONTRIBUTING.md                 pyrightconfig.json
convert_hf_to_gguf.py*          README.md
convert_hf_to_gguf_update.py*   requirements/
convert_llama_ggml_to_gguf.py*  requirements.txt
convert_lora_to_gguf.py*        scripts/
docs/                           SECURITY.md
examples/                       src/
flake.lock                      tests/
flake.nix                       tools/
ggml/                           vendor/


In [10]:
import os
import tarfile
from google.colab import files

# Define source folder and output archive name
source_folder = 'build_t4'
output_archive = 'build_t4_2_1_0.tar.gz'

# Create the tar.gz archive
with tarfile.open(output_archive, "w:gz") as tar:
    tar.add(source_folder, arcname=os.path.basename(source_folder))

print(f"Archive {output_archive} created successfully!")

# Download the archive to your local machine
files.download(output_archive)

Archive build_t4_2_1_0.tar.gz created successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# Verify binaries
print("\n📦 Built binaries:")
print("="*60)
!ls -lh build_t4/bin/llama-server
!ls -lh build_t4/bin/llama-cli
!ls -lh build_t4/bin/llama-quantize

print("\n📚 CUDA libraries:")
print("="*60)
!ls -lh build_t4/bin/libggml-cuda.so* | head -3
!ls -lh build_t4/bin/libllama.so* | head -3

# Test llama-server
print("\n🧪 Testing llama-server...")
#!LD_LIBRARY_PATH=/content/llama.cpp/build_t4/bin:/usr/local/cuda/lib64 \
#    /content/llama.cpp/build_t4/bin/llama-server --version

!LD_LIBRARY_PATH=/content/llama.cpp/build_t4/bin:/usr/local/cuda/lib64:/usr/lib64-nvidia \
    /content/llama.cpp/build_t4/bin/llama-server --version


📦 Built binaries:
-rwxr-xr-x 1 root root 6.7M Jan 15 06:00 build_t4/bin/llama-server
-rwxr-xr-x 1 root root 5.1M Jan 15 06:00 build_t4/bin/llama-cli
-rwxr-xr-x 1 root root 434K Jan 15 05:59 build_t4/bin/llama-quantize

📚 CUDA libraries:
lrwxrwxrwx 1 root root   17 Jan 15 05:50 build_t4/bin/libggml-cuda.so -> libggml-cuda.so.0
lrwxrwxrwx 1 root root   21 Jan 15 05:50 build_t4/bin/libggml-cuda.so.0 -> libggml-cuda.so.0.9.5
-rwxr-xr-x 1 root root 221M Jan 15 05:50 build_t4/bin/libggml-cuda.so.0.9.5
lrwxrwxrwx 1 root root   13 Jan 15 05:52 build_t4/bin/libllama.so -> libllama.so.0
lrwxrwxrwx 1 root root   20 Jan 15 05:52 build_t4/bin/libllama.so.0 -> libllama.so.0.0.7739
-rwxr-xr-x 1 root root 2.9M Jan 15 05:52 build_t4/bin/libllama.so.0.0.7739

🧪 Testing llama-server...
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
version: 7739 (36f013246)
built with GNU 11.4.0 for Linux x86_64


## Step 5: Build llcuda Python Package

---



This creates the llcuda Python package with CUDA backend for Unsloth integration.

In [22]:
pwd

'/content/llcuda'

In [26]:
!pip install -q --upgrade setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [27]:
os.chdir('/content/llcuda')

# Check llcuda version
!cat pyproject.toml | grep "^version"

print("\n🔨 Building llcuda Python package...")
print("   This takes ~2-3 minutes\n")

!python3 -m build --wheel --no-isolation

print("\n✅ llcuda package built")
!ls -lh dist/*.whl

version = "2.1.0"

🔨 Building llcuda Python package...
   This takes ~2-3 minutes

* Getting build dependencies for wheel...
/usr/local/lib/python3.12/dist-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
!!

        ********************************************************************************
        Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).

        By 2026-Feb-18, you need to update your project and remove deprecated calls
        or your builds will no longer be supported.

        See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
        ********************************************************************************

!!
  corresp(dist, value, root_dir)
running egg_info
writing llcuda.egg-info/PKG-INFO
writing dependency_links

## Step 6: Create Unified Binary Package

Creates a single tar file containing:
- llama.cpp binaries (llama-server, llama-cli, etc.)
- CUDA libraries (libggml-cuda.so with FlashAttention)
- llcuda Python wheel
- Installation scripts
- Documentation

In [28]:
os.chdir('/content')

# Create package structure
print("\n📦 Creating unified package structure...")

!rm -rf llcuda-complete-t4
!mkdir -p llcuda-complete-t4/bin
!mkdir -p llcuda-complete-t4/lib
!mkdir -p llcuda-complete-t4/python
!mkdir -p llcuda-complete-t4/docs

# Copy llama.cpp binaries
print("\n📥 Copying llama.cpp binaries...")
!cp llama.cpp/build_t4/bin/llama-server llcuda-complete-t4/bin/
!cp llama.cpp/build_t4/bin/llama-cli llcuda-complete-t4/bin/
!cp llama.cpp/build_t4/bin/llama-quantize llcuda-complete-t4/bin/
!cp llama.cpp/build_t4/bin/llama-embedding llcuda-complete-t4/bin/
!cp llama.cpp/build_t4/bin/llama-bench llcuda-complete-t4/bin/

# Copy CUDA libraries
print("📥 Copying CUDA libraries...")
!cp llama.cpp/build_t4/bin/*.so* llcuda-complete-t4/lib/

# Copy llcuda Python wheel
print("📥 Copying llcuda Python package...")
!cp llcuda/dist/*.whl llcuda-complete-t4/python/

# Copy documentation
!cp llcuda/README.md llcuda-complete-t4/docs/LLCUDA_README.md 2>/dev/null || echo "No README found"

print("\n✅ Package structure created")


📦 Creating unified package structure...

📥 Copying llama.cpp binaries...
📥 Copying CUDA libraries...
📥 Copying llcuda Python package...

✅ Package structure created


In [29]:
# Create installation script
os.chdir('/content/llcuda-complete-t4')

# Use r""" to make this a raw string, preventing Python's SyntaxWarning
install_script = r"""#!/bin/bash
# llcuda Complete Installation Script
# Tesla T4 (SM 7.5) - CUDA 12.x

set -e

echo "===================================================================="
echo "llcuda Complete Installation - Tesla T4 CUDA 12"
echo "===================================================================="
echo ""

# Detect installation directory
INSTALL_DIR="${1:-$HOME/.local/llcuda}"
echo "📁 Installation directory: $INSTALL_DIR"
echo ""

# Create directories
mkdir -p $INSTALL_DIR/bin
mkdir -p $INSTALL_DIR/lib

# Copy binaries
echo "📥 Installing binaries..."
cp bin/* $INSTALL_DIR/bin/
chmod +x $INSTALL_DIR/bin/*

# Copy libraries
echo "📥 Installing libraries..."
cp lib/*.so* $INSTALL_DIR/lib/

# Install Python package
echo "📥 Installing Python package..."
pip install python/*.whl --force-reinstall

# Setup environment
echo ""
echo "===================================================================="
echo "✅ Installation Complete!"
echo "===================================================================="
echo ""
echo "Add these to your ~/.bashrc or session:"
echo ""
echo "export PATH=\"$INSTALL_DIR/bin:\$PATH\""
echo "export LD_LIBRARY_PATH=\"$INSTALL_DIR/lib:\$LD_LIBRARY_PATH\""
echo "export LLAMA_SERVER_PATH=\"$INSTALL_DIR/bin/llama-server\""
echo ""
echo "Quick test:"
echo "  python -c 'import llcuda; print(llcuda.__version__)'"
echo ""
"""

with open('install.sh', 'w') as f:
    f.write(install_script)

!chmod +x install.sh
print("✅ Installation script created without warnings")

✅ Installation script created without warnings


In [30]:
# Create README
readme_content = """# llcuda Complete - CUDA 12 Binaries for Tesla T4

**Version**: 2.0.1
**Built**: Google Colab
**Target**: Tesla T4 (SM 7.5)
**CUDA**: 12.x

## Contents

```
llcuda-complete-t4/
├── bin/                    # Binaries
│   ├── llama-server       # HTTP inference server (6.5 MB)
│   ├── llama-cli          # CLI tool (4.2 MB)
│   ├── llama-quantize     # Model quantization
│   ├── llama-embedding    # Embedding generation
│   └── llama-bench        # Benchmarking
├── lib/                    # CUDA Libraries
│   ├── libggml-cuda.so*   # CUDA kernels with FlashAttention (174 MB)
│   ├── libggml-base.so*   # Base GGML (721 KB)
│   ├── libggml-cpu.so*    # CPU fallback (1.1 MB)
│   ├── libllama.so*       # Llama.cpp core (2.9 MB)
│   └── libmtmd.so*        # Multi-threading (877 KB)
├── python/                 # Python Package
│   └── llcuda-*.whl       # llcuda wheel (~70 KB)
├── docs/                   # Documentation
│   └── LLCUDA_README.md
├── install.sh              # Installation script
└── README.md               # This file
```

## Features

✅ **FlashAttention 2** - 2-3x faster for long contexts
✅ **Tensor Core Optimization** - FP16/INT8 acceleration
✅ **CUDA Graphs** - Reduced kernel launch overhead
✅ **All Quantization Types** - Q2_K through Q8_0, NF4
✅ **Unsloth Integration** - Backend for Unsloth fine-tuned models

## Quick Installation

```bash
# Extract
tar -xzf llcuda-complete-cuda12-t4.tar.gz
cd llcuda-complete-t4

# Install (default: ~/.local/llcuda)
bash install.sh

# Or install to custom directory
bash install.sh /your/custom/path
```

## Manual Installation

```bash
# Copy binaries
cp bin/* ~/.local/bin/
cp lib/*.so* ~/.local/lib/

# Install Python package
pip install python/*.whl

# Add to environment
export PATH="$HOME/.local/bin:$PATH"
export LD_LIBRARY_PATH="$HOME/.local/lib:$LD_LIBRARY_PATH"
export LLAMA_SERVER_PATH="$HOME/.local/bin/llama-server"
```

## Usage Examples

### 1. Using llcuda with Unsloth GGUF Models

```python
import llcuda

# Initialize engine
engine = llcuda.InferenceEngine()

# Load Unsloth GGUF model
engine.load_model("unsloth/gemma-3-1b-it-GGUF:gemma-3-1b-it-Q4_K_M.gguf")

# Run inference
result = engine.infer("What is AI?", max_tokens=100)
print(result.text)
print(f"Speed: {result.tokens_per_sec:.1f} tok/s")  # ~45 tok/s on T4
```

### 2. Using llama-server Directly

```bash
# Download model
wget https://huggingface.co/unsloth/gemma-3-1b-it-GGUF/resolve/main/gemma-3-1b-it-Q4_K_M.gguf

# Start server
llama-server \
    --model gemma-3-1b-it-Q4_K_M.gguf \
    --n-gpu-layers 99 \
    --ctx-size 2048 \
    --port 8090
```

### 3. Model Registry (Built-in Models)

```python
import llcuda

engine = llcuda.InferenceEngine()

# Use model registry name
engine.load_model("gemma-3-1b-Q4_K_M")  # Auto-downloads

result = engine.infer("Hello!", max_tokens=50)
print(result.text)
```

## Performance (Tesla T4)

| Model | Speed | VRAM | Context |
|-------|-------|------|--------|
| Gemma 3-1B Q4_K_M | 45 tok/s | 1.2 GB | 2048 |
| Llama 3.2-3B Q4_K_M | 30 tok/s | 2.0 GB | 4096 |
| Qwen 2.5-7B Q4_K_M | 18 tok/s | 5.0 GB | 8192 |
| Llama 3.1-8B Q4_K_M | 15 tok/s | 5.5 GB | 8192 |

## Requirements

- **GPU**: Tesla T4 (SM 7.5) or compatible
- **CUDA**: 12.x runtime
- **Python**: 3.11+
- **OS**: Linux x86_64

## Build Information

- **llama.cpp**: Built with SM 7.5, FlashAttention, CUDA Graphs
- **GGML**: Version 0.9.5
- **llcuda**: Version 2.0.1
- **Built on**: Google Colab
- **Compiler**: GCC + nvcc (CUDA 12)

## Troubleshooting

### Library not found
```bash
export LD_LIBRARY_PATH="$HOME/.local/lib:/usr/local/cuda/lib64:$LD_LIBRARY_PATH"
```

### llama-server: libcuda.so.1 not found
```bash
# Find libcuda.so.1
find /usr -name "libcuda.so.1" 2>/dev/null

# Add to LD_LIBRARY_PATH
export LD_LIBRARY_PATH="/path/to/cuda/lib:$LD_LIBRARY_PATH"
```

### Import error in Python
```bash
pip install --force-reinstall python/*.whl
```

## Links

- **llcuda GitHub**: https://github.com/waqasm86/llcuda
- **llama.cpp**: https://github.com/ggerganov/llama.cpp
- **Unsloth**: https://github.com/unslothai/unsloth
- **PyPI**: https://pypi.org/project/llcuda/

## License

MIT License - Same as llama.cpp and llcuda

---

**Built with ❤️ for Unsloth + llcuda integration on Tesla T4**
"""

with open('README.md', 'w') as f:
    f.write(readme_content)

print("✅ README created")

✅ README created


In [31]:
# Create metadata file
import datetime

metadata = f"""# Build Metadata

Build Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}
Platform: Google Colab
GPU: Tesla T4 (SM 7.5)
CUDA: 12.x
Python: {sys.version.split()[0]}

## Components

llama.cpp:
  - Branch: main
  - Features: FlashAttention, CUDA Graphs, Tensor Cores
  - Compute: SM 7.5

llcuda:
  - Version: 2.0.1
  - Repository: https://github.com/waqasm86/llcuda

## Build Options

CMAKE_BUILD_TYPE: Release
GGML_CUDA: ON
GGML_CUDA_FA: ON (FlashAttention)
GGML_CUDA_FA_ALL_QUANTS: ON
GGML_CUDA_GRAPHS: ON
CMAKE_CUDA_ARCHITECTURES: 75
BUILD_SHARED_LIBS: ON
"""

with open('BUILD_INFO.txt', 'w') as f:
    f.write(metadata)

print("✅ Metadata file created")

✅ Metadata file created


In [32]:
# Show package contents and sizes
os.chdir('/content/llcuda-complete-t4')

print("\n" + "="*70)
print("📦 Package Contents")
print("="*70)

!echo "\n📁 Directory structure:"
!tree -L 2 -h || find . -maxdepth 2 -type f -exec ls -lh {} \;

!echo "\n💾 Size breakdown:"
!du -sh .
!du -sh bin/
!du -sh lib/
!du -sh python/

!echo "\n📊 File counts:"
!echo "Binaries: $(ls bin/ | wc -l)"
!echo "Libraries: $(ls lib/*.so* | wc -l)"
!echo "Python packages: $(ls python/*.whl | wc -l)"


📦 Package Contents
\n📁 Directory structure:
/bin/bash: line 1: tree: command not found
-rw-r--r-- 1 root root 4.3K Jan 15 06:37 ./README.md
-rwxr-xr-x 1 root root 1.4K Jan 15 06:37 ./install.sh
-rw-r--r-- 1 root root 87K Jan 15 06:37 ./python/llcuda-2.1.0-py3-none-any.whl
-rwxr-xr-x 1 root root 721K Jan 15 06:37 ./lib/libggml-base.so.0
-rwxr-xr-x 1 root root 949K Jan 15 06:37 ./lib/libggml-cpu.so.0.9.5
-rwxr-xr-x 1 root root 2.9M Jan 15 06:37 ./lib/libllama.so.0
-rwxr-xr-x 1 root root 54K Jan 15 06:37 ./lib/libggml.so.0.9.5
-rwxr-xr-x 1 root root 721K Jan 15 06:37 ./lib/libggml-base.so
-rwxr-xr-x 1 root root 894K Jan 15 06:37 ./lib/libmtmd.so.0.0.7739
-rwxr-xr-x 1 root root 54K Jan 15 06:37 ./lib/libggml.so.0
-rwxr-xr-x 1 root root 949K Jan 15 06:37 ./lib/libggml-cpu.so.0
-rwxr-xr-x 1 root root 2.9M Jan 15 06:37 ./lib/libllama.so
-rwxr-xr-x 1 root root 894K Jan 15 06:37 ./lib/libmtmd.so
-rwxr-xr-x 1 root root 54K Jan 15 06:37 ./lib/libggml.so
-rwxr-xr-x 1 root root 2.9M Jan 15 06:37 .

## Step 7: Create Tar Archive

In [33]:
os.chdir('/content')

print("\n📦 Creating tar archive...")
print("   This may take 2-3 minutes...\n")

!tar -czf llcuda-complete-cuda12-t4.tar.gz llcuda-complete-t4/

print("\n✅ Tar archive created!")
print("\n" + "="*70)
!ls -lh llcuda-complete-cuda12-t4.tar.gz
!du -h llcuda-complete-cuda12-t4.tar.gz
print("="*70)


📦 Creating tar archive...
   This may take 2-3 minutes...


✅ Tar archive created!

-rw-r--r-- 1 root root 267M Jan 15 06:38 llcuda-complete-cuda12-t4.tar.gz
267M	llcuda-complete-cuda12-t4.tar.gz


In [34]:
# Generate SHA256 checksum
!sha256sum llcuda-complete-cuda12-t4.tar.gz > llcuda-complete-cuda12-t4.tar.gz.sha256
!cat llcuda-complete-cuda12-t4.tar.gz.sha256

print("\n✅ Checksum generated")

953b612edcd3b99b66ae169180259de19a6ef5da1df8cdcacbc4b09fd128a5dd  llcuda-complete-cuda12-t4.tar.gz

✅ Checksum generated


## Step 8: Download Package

Download the unified tar file to your local machine.

In [35]:
from google.colab import files

print("\n" + "="*70)
print("📥 DOWNLOAD READY")
print("="*70)
print("\nFile: llcuda-complete-cuda12-t4.tar.gz")
print("Size: ~350-400 MB (compressed)")
print("\nDownloading...\n")

files.download('/content/llcuda-complete-cuda12-t4.tar.gz')

print("\n✅ Download started!")
print("\nOptionally download checksum:")
files.download('/content/llcuda-complete-cuda12-t4.tar.gz.sha256')


📥 DOWNLOAD READY

File: llcuda-complete-cuda12-t4.tar.gz
Size: ~350-400 MB (compressed)

Downloading...



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Download started!

Optionally download checksum:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import os
print(os.path.exists('/content/llcuda-complete-cuda12-t4.tar.gz'))
print(os.path.exists('/content/llcuda-complete-cuda12-t4.tar.gz.sha256'))

True
True


In [39]:
ls

llama.cpp/  llcuda-complete-cuda12-t4.tar.gz         llcuda-complete-t4/
llcuda/     llcuda-complete-cuda12-t4.tar.gz.sha256  sample_data/


In [43]:
#from google.colab import files
#files.download('/content/llcuda-complete-cuda12-t4.tar.gz')
#files.download('/content/llcuda-complete-cuda12-t4.tar.gz.sha256')

In [44]:
import torch
print(torch.cuda.is_available())

True


In [45]:
cuda_info = llcuda.detect_cuda()
print(f"\nCUDA available: {cuda_info['available']}")
if cuda_info['available'] and cuda_info['gpus']:
    print(f"GPU: {cuda_info['gpus'][0]['name']}")
    print(f"Compute: SM {cuda_info['gpus'][0]['compute_capability']}")
else:
    print("No CUDA GPU detected or available.")


CUDA available: False
No CUDA GPU detected or available.


## Step 9: Quick Test (Optional)
**bold text**
Test the build directly in Colab before downloading.

In [36]:
# Install llcuda from the built wheel
!pip install -q /content/llcuda-complete-t4/python/*.whl --force-reinstall

import llcuda
print(f"\n✅ llcuda version: {llcuda.__version__}")

# Set environment
import os
os.environ['LD_LIBRARY_PATH'] = '/content/llcuda-complete-t4/lib:/usr/local/cuda/lib64'
os.environ['LLAMA_SERVER_PATH'] = '/content/llcuda-complete-t4/bin/llama-server'

# Test GPU detection
cuda_info = llcuda.detect_cuda()
print(f"\nCUDA available: {cuda_info['available']}")
print(f"GPU: {cuda_info['gpus'][0]['name']}")
print(f"Compute: SM {cuda_info['gpus'][0]['compute_capability']}")

print("\n✅ llcuda works!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.5/534.5 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 134.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.9/152.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.8/201.8 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5

/usr/local/lib/python3.12/dist-packages/llcuda/__init__.py:112: RuntimeWarning: llcuda bootstrap failed: cannot import name '__version__' from partially initialized module 'llcuda' (most likely due to a circular import) (/usr/local/lib/python3.12/dist-packages/llcuda/__init__.py)
Some features may not work. Please check your installation.
  warnings.warn(


llcuda: New version available (2.0.6) - pip install --upgrade git+https://github.com/llcuda/llcuda.git

✅ llcuda version: 2.1.0

CUDA available: False


IndexError: list index out of range

## 🎉 Build Complete!

### What You Have

**File**: `llcuda-complete-cuda12-t4.tar.gz` (~350-400 MB)

**Contents**:
- ✅ llama.cpp binaries (llama-server with FlashAttention)
- ✅ CUDA libraries (libggml-cuda.so, etc.)
- ✅ llcuda Python package (wheel)
- ✅ Installation script
- ✅ Documentation

### Next Steps

1. **Extract on target system**:
   ```bash
   tar -xzf llcuda-complete-cuda12-t4.tar.gz
   cd llcuda-complete-t4
   bash install.sh
   ```

2. **Use with Unsloth**:
   ```python
   import llcuda
   engine = llcuda.InferenceEngine()
   engine.load_model("gemma-3-1b-Q4_K_M")
   result = engine.infer("Hello!", max_tokens=50)
   print(result.text)
   ```

3. **Upload to GitHub Releases** (optional):
   ```bash
   gh release upload v2.0.1 llcuda-complete-cuda12-t4.tar.gz
   ```

### Features

✅ Tesla T4 optimized (SM 7.5)
✅ FlashAttention enabled (2-3x faster)
✅ Tensor Core optimization
✅ CUDA 12.x compatible
✅ All quantization formats
✅ Unsloth integration ready

---

**Built with**: Google Colab | Tesla T4 | CUDA 12 | Python 3.10+
**For**: llcuda v2.0.1 - Unsloth CUDA Backend


In [ ]:
!echo $LD_LIBRARY_PATH

/usr/lib64-nvidia


In [ ]:
# Test llama-server with corrected library path
print("\n🧪 Testing llama-server...")
!LD_LIBRARY_PATH=/content/llama.cpp/build_t4/bin:/usr/local/cuda/lib64:/usr/lib64-nvidia \
    /content/llama.cpp/build_t4/bin/llama-server --version


🧪 Testing llama-server...
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
version: 7662 (56d2fed2b)
built with GNU 11.4.0 for Linux x86_64


In [ ]:
# Verify binaries
print("\n📦 Built binaries:")
print("="*60)
!ls -lh build_t4/bin/llama-server
!ls -lh build_t4/bin/llama-cli
!ls -lh build_t4/bin/llama-quantize

print("\n📚 CUDA libraries:")
print("="*60)
!ls -lh build_t4/bin/libggml-cuda.so* | head -3
!ls -lh build_t4/bin/libllama.so* | head -3


📦 Built binaries:
-rwxr-xr-x 1 root root 6.5M Jan  7 16:40 build_t4/bin/llama-server
-rwxr-xr-x 1 root root 4.2M Jan  7 16:39 build_t4/bin/llama-cli
-rwxr-xr-x 1 root root 434K Jan  7 16:39 build_t4/bin/llama-quantize

📚 CUDA libraries:
lrwxrwxrwx 1 root root   17 Jan  7 16:30 build_t4/bin/libggml-cuda.so -> libggml-cuda.so.0
lrwxrwxrwx 1 root root   21 Jan  7 16:30 build_t4/bin/libggml-cuda.so.0 -> libggml-cuda.so.0.9.5
-rwxr-xr-x 1 root root 221M Jan  7 16:30 build_t4/bin/libggml-cuda.so.0.9.5
lrwxrwxrwx 1 root root   13 Jan  7 16:32 build_t4/bin/libllama.so -> libllama.so.0
lrwxrwxrwx 1 root root   20 Jan  7 16:32 build_t4/bin/libllama.so.0 -> libllama.so.0.0.7662
-rwxr-xr-x 1 root root 2.9M Jan  7 16:32 build_t4/bin/libllama.so.0.0.7662


In [ ]:
# 1. Update LD_LIBRARY_PATH to include Nvidia drivers
import os
os.environ['LD_LIBRARY_PATH'] = '/content/llcuda-complete-t4/lib:/usr/local/cuda/lib64:/usr/lib64-nvidia'
os.environ['LLAMA_SERVER_PATH'] = '/content/llcuda-complete-t4/bin/llama-server'

# 2. Force reinstall the local wheel again to ensure correct version
!pip install -q /content/llcuda-complete-t4/python/*.whl --force-reinstall

import llcuda
import importlib
importlib.reload(llcuda) # Ensure we are using the freshly installed version

print(f"\n✅ llcuda version: {llcuda.__version__}")

# 3. Test GPU detection
cuda_info = llcuda.detect_cuda()
print(f"\nCUDA available: {cuda_info['available']}")

if cuda_info['available'] and len(cuda_info['gpus']) > 0:
    print(f"GPU: {cuda_info['gpus'][0]['name']}")
    print(f"Compute: SM {cuda_info['gpus'][0]['compute_capability']}")
    print("\n✅ llcuda is correctly linked to Tesla T4!")
else:
    print("\n❌ Error: GPU still not detected. Check !ls -l /usr/lib64-nvidia")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.4.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.12.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.4.0 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.4.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, bu

In [ ]:
# 1. Update LD_LIBRARY_PATH (Crucial for T4 drivers)
import os
os.environ['LD_LIBRARY_PATH'] = '/content/llcuda-complete-t4/lib:/usr/local/cuda/lib64:/usr/lib64-nvidia'
os.environ['LLAMA_SERVER_PATH'] = '/content/llcuda-complete-t4/bin/llama-server'

# 2. Reinstall with a pinned NumPy to satisfy most dependencies
# We use --no-warn-conflicts to keep the output clean
!pip install -q /content/llcuda-complete-t4/python/*.whl "numpy<2.2" --force-reinstall --no-warn-conflicts

# 3. Force Python to see the NEW version without restarting the runtime
import sys
import importlib
if 'llcuda' in sys.modules:
    importlib.reload(sys.modules['llcuda'])
import llcuda

print(f"🎯 Targeted Version: v2.0.1")
print(f"✅ Currently Loaded: {llcuda.__version__}")

# 4. Final Verification
cuda_info = llcuda.detect_cuda()
if cuda_info['available']:
    gpu = cuda_info['gpus'][0]
    print(f"\n🚀 GPU Status: {gpu['name']} is READY")
    print(f"📊 Compute Capability: {gpu['compute_capability']}")
    print(f"🔗 Link Status: Success")
else:
    print("\n❌ Error: GPU detection failed.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 117.0 MB/s eta 0:00:00
llcuda: New version available (2.0.1) - pip install --upgrade llcuda
🎯 Targeted Version: v2.0.1
✅ Currently Loaded: 1.2.2

🚀 GPU Status: Tesla T4 is READY
📊 Compute Capability: 7.5
🔗 Link Status: Success
